In [ ]:
!pip install qiskit

In [ ]:
import qiskit
# Enable your account on Qiskit, replace 'My_API_Token' with your newly generated token
qiskit.IBMQ.save_account('ac05e244cf4e53e3842a230becc884bcd7395d0ebc42484e158085d7f8e6db097e404e65eea81c45dce182e8884414201434a248566c9ad7f30d8fb5629b4ceb')
qiskit.IBMQ.load_account()
# After loading credentials we query the backends
# qiskit.IBMQ.backend()

configrc.store_credentials:WARNING:2021-07-12 05:13:35,406: Credentials already present. Set overwrite=True to overwrite.
ibmqfactory.load_account:WARNING:2021-07-12 05:13:35,730: Credentials are already in use. The existing account in the session will be replaced.


<AccountProvider for IBMQ(hub='ibm-q', group='open', project='main')>

In [ ]:
%matplotlib inline
import copy
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()
# svism = Aer.get_backend('statevector_simulator')
# backend = 'simulator_statevetor'
# backend

ibmqfactory.load_account:WARNING:2021-07-12 05:13:38,246: Credentials are already in use. The existing account in the session will be replaced.


In [ ]:
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit
from qiskit.quantum_info.operators import Operator, Pauli
from qiskit.extensions import RXGate, XGate, CXGate, ZGate, IGate

from numpy import pi, sqrt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
from qiskit import Aer
from qiskit.providers.aer import AerSimulator

backend = Aer.get_backend('qasm_simulator')

In [ ]:
from qiskit.test.mock import FakeRueschlikon, FakeTokyo
device_backend = FakeTokyo()
device_backend_2 = FakeRueschlikon()


In [ ]:
backend = AerSimulator.from_backend(device_backend)

In [ ]:
parameters = {'label': 'fermi-hubbard model', 'time': 0.1, 'step': 1, 'chain': 'xxz-chain-potential'}
alpha = {'theta_x': 0,'theta_y': 0, 'theta_z': 0, 'theta_rzz':0, 'gamma_1':0, 'gamma_2': 0 ,'gamma_3': 0}

In [ ]:
num_qubits = 2

#Defining Registers for Suzuki-Trotter Simulation
# qreg_q = QuantumRegister(num_qubits, 'q')
# creg_c = ClassicalRegister(num_qubits, 'c')


#Defining Registers for VFF
qreg_q = QuantumRegister(num_qubits*2, 'q')
creg_c = ClassicalRegister(num_qubits*2, 'c')

def circuit2():
  creg_c = ClassicalRegister(2, 'c')
  circuit_2 = QuantumCircuit(qreg_q, creg_c)
  return circuit_2
# circuit = QuantumCircuit(qreg_q, creg_c)
#Defining Registers for HST (i.e. F(T))
# qreg_q_3 = QuantumRegister(num_qubits*2, 'q_3')
# creg_c_3 = ClassicalRegister(num_qubits*2, 'c_3')
def circuit3():
  circuit_3 = QuantumCircuit(qreg_q, creg_c)
  return circuit_3

In [ ]:
#Domain-Wall State preparation
def domain_wall_state(circuit, num_qubits):
    for i in range(int(num_qubits/2)):
        circuit.x(qreg_q[i])
    return circuit
#Neel-State Prepapration
def neel_state_preparation(circuit, num_qubits):
    for i in range(num_qubits):
        if i%2 == 0:
            circuit.x(qreg_q[i])
    return circuit

In [ ]:
#Magnetization operator
magnetization = Operator(ZGate())
print('magnetization operator:\n',magnetization.data)

#N_half operator
n_half = np.add(0.5*Operator(ZGate()).data,0.5*Operator(IGate()).data)
print('\nn_half operator:\n',n_half)

#spin-correlator operator


magnetization operator:
 [[ 1.+0.j  0.+0.j]
 [ 0.+0.j -1.+0.j]]

n_half operator:
 [[1.+0.j 0.+0.j]
 [0.+0.j 0.+0.j]]


In [ ]:
#XX-chain parameter U = 0 and h = 0 non-interacting molecule
def XX_chain_parameters():
    range_J = 10
    U = 0.0
    h = 0.0
    J = np.array([i for i in range(range_J)])
    return (J,U,h)
# print(XX_chain_parameters())
##Disordered XX-chain U=0 and h = [-h,h](randomly sampled from this set)
def XX_disorder_chain_parameters():
    range_J = 10
    U = 0.0
    h = np.concatenate((np.random.random_sample((5,))-1,np.random.random_sample((5,))), axis = None)
    J = np.array([i+1 for i in range(range_J)])
    return (J,U,h)
#XXZ-spin chain  case of Hisenberg model for quantum magnetism
# U != 0 , h = 0
def XXZ_spin_chain_parameters():
    range_J = 10
    range_U = 5
    h = 0.0
    J = np.array([i for i in range(range_J)])
    U = np.array([i for i in range(range_U)])
    return (J,U,h)
#XXZ- chain with linear potential U>=0, h = h*j
def XXZ_chain_potential_parameters(num_qubits):
    range_J = 10
    range_U = 5
    J = np.array([i for i in range(range_J)])
    U = np.array([i for i in range(range_U)])
    h = np.concatenate((np.random.random_sample((5,))-1,np.random.random_sample((5,))), axis = None)
    for j in range(num_qubits):
        h[j] = h[j]*j
    
    return (J,U,h)

In [ ]:

def circuit_a_j(circuit,qubit_index, h, time):
    circuit.barrier()
    circuit.rz((h*time)/2,qreg_q[qubit_index])
    circuit.barrier()
    return circuit
def circuit_b_j(circuit, qubit_index, U, J, time ):
    #circuit for sigma_zz
    circuit.barrier()
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1])
    circuit.rz((U*time)/2, qreg_q[qubit_index+1])
    circuit.cx(qreg_q[qubit_index],qreg_q[qubit_index+1])
    circuit.barrier()
    #circuit for sigma_yy
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1])
    circuit.cu3((J*time)/2, -pi/2, pi/2, qreg_q[qubit_index+1], qreg_q[qubit_index] )
    circuit.x(qreg_q[qubit_index+1] )
    circuit.cu3((J*time)/2, pi/2, -pi/2, qreg_q[qubit_index+1], qreg_q[qubit_index])
    circuit.x(qreg_q[qubit_index+1] )
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1] )
    #circuit for sigma_xx
    circuit.barrier()
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1])
    circuit.u3((-J*time)/2, -pi/2, pi/2, qreg_q[qubit_index])
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1])
    circuit.barrier()
    return circuit
def circuit_c_j(circuit, qubit_index, U, J, time):
    #circuit for sigma_zz
    circuit.barrier()
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1])
    circuit.rz((U*time), qreg_q[qubit_index+1])
    circuit.cx(qreg_q[qubit_index],qreg_q[qubit_index+1])
    circuit.barrier()
    #circuit for sigma_yy
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1])
    circuit.cu3((J*time), -pi/2, pi/2, qreg_q[qubit_index+1], qreg_q[qubit_index] )
    circuit.x(qreg_q[qubit_index+1] )
    circuit.cu3((J*time), pi/2, -pi/2, qreg_q[qubit_index+1], qreg_q[qubit_index])
    circuit.x(qreg_q[qubit_index+1] )
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1] )
    #circuit for sigma_xx
    circuit.barrier()
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1])
    circuit.u3((-J*time), -pi/2, pi/2, qreg_q[qubit_index])
    circuit.cx(qreg_q[qubit_index], qreg_q[qubit_index+1])
    circuit.barrier()
    return circuit

# circuit.draw('mpl')

In [ ]:
def circuit_for_magnetization(circuit, time, step, num_qubits):
    for j in range(step):
        for i in range(num_qubits):
            J,U,h = XX_chain_parameters()
            circuit_a_j(circuit, i, h, time)
            if i%2 == 0:
                circuit_c_j(circuit, i, U, J[1], time)
            elif i%2 != 0 and i != num_qubits - 1 :
                circuit_b_j(circuit, i, U, J[1], time)
            circuit_a_j(circuit, i, h, time)
    return circuit

In [ ]:
def circuit_for_n_half(circuit, time, step, num_qubits, chain):
    for j in range(step):
        if chain == 'xx-disordered':
            for i in range(num_qubits):
                J,U,h = XX_disorder_chain_parameters()
                circuit_a_j(circuit, i, h[i], time)
                if i%2 == 0:
                    circuit_c_j(circuit, i, U, J[1], time)
                elif i%2 != 0 and i != num_qubits - 1:
                    circuit_b_j(circuit, i, U, J[1], time)
                circuit_a_j(circuit, i, h, time)
        elif chain == 'xxz-spin-chain':
            for i in range(num_qubits):
                J,U,h = XXZ_spin_chain_parameters()
                circuit_a_j(circuit, i, h, time)
                if i%2 == 0:
                    circuit_c_j(circuit, i, U[2], J[1], time)
                elif i%2 != 0 and i != num_qubits - 1:
                    circuit_b_j(circuit, i, U[2], J[1], time)
                circuit_a_j(circuit, i, h, time)
        elif chain == 'xxz-chain-potential':
            for i in range(num_qubits):
                J,U,h = XXZ_chain_potential_parameters(num_qubits)
                circuit_a_j(circuit, i, h[i], time)
                if i%2 == 0:
                    circuit_c_j(circuit, i, U[2], J[1], time)
                elif i%2 != 0 and i != num_qubits - 1:
                    circuit_b_j(circuit, i, U[2], J[1], time)
                circuit_a_j(circuit, i, h[i], time)
    return circuit        
        

In [ ]:
def ansatz_for_W(circuit_2, num_qubits, alpha):

  for i in range(num_qubits):
    ####### First Layer of W(theta) ######
        
    circuit_2.rx(alpha['theta_x'],qreg_q[num_qubits + i])
    circuit_2.ry(alpha['theta_y'],qreg_q[num_qubits + i])
    circuit_2.rz(alpha['theta_z'],qreg_q[num_qubits + i])
  for i in range(num_qubits):

    ##### Second Sub-Layer for W(theta) ########
    if i % 2 != 0 :
      if i == num_qubits - 1:
        circuit_2.rzz(alpha['theta_rzz'], qreg_q[num_qubits + i], qreg_q[num_qubits])
      else:
        circuit_2.rzz(alpha['theta_rzz'], qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])

    ##### Third Sub-Layer for W(theta) ########
    elif i % 2 == 0 :
      if i == num_qubits - 1:
        circuit_2.rzz(alpha['theta_rzz'], qreg_q[num_qubits + i], qreg_q[num_qubits])
      else:
        circuit_2.rzz(alpha['theta_rzz'], qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])
  for i in range(num_qubits):

    ##### Final Single-Qubit layer of W(theta)####
    circuit_2.rx(alpha['theta_x'],qreg_q[num_qubits + i])
    circuit_2.ry(alpha['theta_y'],qreg_q[num_qubits + i])
    circuit_2.rz(alpha['theta_z'],qreg_q[num_qubits + i])

  return circuit_2

In [ ]:
def ansatz_for_W_conjugate(circuit_2, num_qubits, alpha):
  for i in range(num_qubits):

    ###### First Sub-Layer for W(theta)_conjugate ########
    circuit_2.rx(-alpha['theta_x'],qreg_q[num_qubits + i])
    circuit_2.ry(-alpha['theta_y'], qreg_q[num_qubits + i])
    circuit_2.rz(-alpha['theta_z'],qreg_q[num_qubits + i])
  for i in range(num_qubits):

    ##### Second Sub-Layer for W(theta)_conjugate ########
    if i % 2 != 0:
      if i == num_qubits - 1:
        circuit_2.rzz(-alpha['theta_rzz'],qreg_q[num_qubits + i], qreg_q[num_qubits])
      else:
        circuit_2.rzz(-alpha['theta_rzz'],qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])

    ##### Third Sub-Layer for W(theta)_conjugate ########
    elif i % 2 == 0:
      if i == num_qubits - 1:
        circuit_2.rzz(-alpha['theta_rzz'], qreg_q[num_qubits + i], qreg_q[num_qubits])
      else:
        circuit_2.rzz(-alpha['theta_rzz'], qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])
  for i in range(num_qubits):

    ##### Final Single-Qubit layer of W(theta)_conjugate ####
    circuit_2.rz(-alpha['theta_z'],qreg_q[num_qubits + i])
    circuit_2.ry(-alpha['theta_y'], qreg_q[num_qubits + i])
    circuit_2.rx(-alpha['theta_x'],qreg_q[num_qubits + i])

  return circuit_2

In [ ]:
def ansatz_for_D(circuit_2, num_qubits, alpha):
  for i in range(num_qubits):

    ##### Firt Sub- Layer for D(alpha['gamma'], T) #######
          
    circuit_2.rz(alpha['gamma_3'], qreg_q[num_qubits + i])
    circuit_2.z(qreg_q[num_qubits + i])
    circuit_2.z(qreg_q[num_qubits + i])
  for i in range(num_qubits):

    ##### Second Sub - Layer for D(alpha['gamma', T]) ######
    if i == num_qubits - 1:
      circuit_2.rzz(alpha['gamma_2'], qreg_q[num_qubits + i], qreg_q[num_qubits])
    else:
      circuit_2.rzz(alpha['gamma_2'], qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])
  for i in range(num_qubits):
    
    ######## Third Sub-Layer for D(alpha['gamma'], T) ########
    
    
    circuit_2.rz(alpha['gamma_1'], qreg_q[num_qubits + i])

  return circuit_2


In [ ]:
def entanglement_fidelity(circuit_2, num_qubits, parameters, alpha):
    label = parameters['label']
    for i in range(num_qubits):
        circuit_2.h(qreg_q[i])
        circuit_2.cx(qreg_q[i], qreg_q[num_qubits + i])
    #unitary creation from suzuki-trotterization
    # qreg_q = QuantumRegister(num_qubits, 'q')
    # creg_c = ClassicalRegister(num_qubits, 'c')
    # temp_circuit = QuantumCircuit(qreg_q, creg_c)

    if label == 'magnetization':

        circuit_for_magnetization(circuit_2, parameters['time'], parameters['step'], num_qubits)
        # circuit_2.draw('text', scale = 0.25, plot_barriers=False, fold=-1)
    elif label == 'n_half':
        circuit_for_n_half(circuit_2, parameters['time'], parameters['step'], num_qubits, parameters['chain'])
    elif label == 'spin-correlator':
        pass
    elif label == 'quantum-fisher-information':
        pass
    elif label == 'fermi-hubbard model':
      circuit_2.u3(pi, pi, 0, qreg_q[0])
      circuit_2.u3(pi, pi, 0, qreg_q[1])
      circuit_2.u1(0, qreg_q[0])
      circuit_2.u1(0, qreg_q[1])
    #ansastz creation for V_conjugate
    # for i in range(num_qubits):

    ###### W(theta)_dagger_conjugate  #######
    ansatz_for_W(circuit_2, num_qubits, alpha)

    ###### D(alpha['gamma'])_conjugate #######
    alpha_D = copy.deepcopy(alpha)
    alpha_D['gamma_1'] = -alpha_D['gamma_1']
    alpha_D['gamma_2'] = -alpha_D['gamma_2']
    alpha_D['gamma_3'] = -alpha_D['gamma_3']
    ansatz_for_D(circuit_2, num_qubits, alpha_D)

    ###### W(theta)_conjugate ########
    alpha_W_conjugate = copy.deepcopy(alpha)
    alpha_W_conjugate['theta_x'] = -alpha_W_conjugate['theta_x']
    alpha_W_conjugate['theta_y'] = -alpha_W_conjugate['theta_y']
    alpha_W_conjugate['theta_z'] = -alpha_W_conjugate['theta_z']
    alpha_W_conjugate['theta_rzz'] = -alpha_W_conjugate['theta_rzz']
    ansatz_for_W_conjugate(circuit_2, num_qubits, alpha_W_conjugate)

        # ####### First Layer of W(theta)_dagger_conjugate ######

        # circuit_2.x(qreg_q[num_qubits + i])
        # circuit_2.y(qreg_q[num_qubits + i])
        # circuit_2.z(qreg_q[num_qubits + i])
        
        # ##### Second Sub-Layer for W(theta)_dagger_conjugate ########
        # if i % 2 != 0:
        #     circuit_2.cx(qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])
        # ##### Third Sub-Layer for W(theta)_dagger_conjugate ########
        # elif i % 2 == 0:
        #     circuit_2.rzz(alpha['theta'], qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])

        # ##### Final Single-Qubit layer of W(theta)_dagger_conjugate ####
        # circuit_2.z(qreg_q[num_qubits + i])
        # circuit_2.y(qreg_q[num_qubits + i])
        # circuit_2.x(qreg_q[num_qubits + i])
    
        # ##### Firt Sub- Layer for D(alpha['gamma'], T)_conjugate #######
        
        # circuit_2.rz(-alpha['gamma'], qreg_q[num_qubits + i])
        # circuit_2.z(qreg_q[num_qubits + i])
        # circuit_2.z(qreg_q[num_qubits + i])
        
        # ##### Second Sub - Layer for D(alpha['gamma', T])_conjugate ######
        
        # circuit_2.rzz(-alpha['gamma'], qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])
        
        # ######## Third Sub-Layer for D(alpha['gamma'], T)_conjugate ########
        
        
        # circuit_2.rz(-alpha['gamma'], qreg_q[num_qubits + i])
        
        # ###### First Sub-Layer for W(theta)_conjugate ########
        # circuit_2.x(qreg_q[num_qubits + i])
        # circuit_2.u3(pi/2, 5*pi/2, 5*pi/2, qreg_q[num_qubits + i])
        # circuit_2.z(qreg_q[num_qubits + i])
        # ##### Second Sub-Layer for W(theta)_conjugate ########
        # if i % 2 != 0:
        #     circuit_2.cx(qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])
        # ##### Third Sub-Layer for W(theta)_conjugate ########
        # elif i % 2 == 0:
        #     circuit_2.rzz(-alpha['theta'], qreg_q[num_qubits + i], qreg_q[num_qubits + (i+1)])

        # ##### Final Single-Qubit layer of W(theta)_conjugate ####
        #     circuit_2.z(qreg_q[num_qubits + i])
        #     circuit_2.u3(pi/2, 5*pi/2, 5*pi/2, qreg_q[num_qubits + i])
        #     circuit_2.x(qreg_q[num_qubits + i])
            
    return circuit_2
    
        

In [ ]:
x = entanglement_fidelity(circuit2(), num_qubits, parameters, alpha)
x.draw('text', scale = 0.25, plot_barriers = False, fold = -1)

┌───┐     ┌───────────┐  ┌───────┐                                                                                                                                                                                 
q_0: ┤ H ├──■──┤ U3(π,π,0) ├──┤ U1(0) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     ├───┤  │  └───────────┘┌─┴───────┴─┐┌───────┐                                                                                                                                                                      
q_1: ┤ H ├──┼────────■──────┤ U3(π,π,0) ├┤ U1(0) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
     └───┘┌─┴─┐      │      └─┬───────┬─┘├───────┤┌───────┐                ┌───────┐┌───────┐┌───────┐┌───────┐┌───┐┌───┐                ┌───────┐┌───────┐┌───────┐┌───────┐                ┌───────┐┌───────┐┌───────┐
q_2: ─────┤ X ├──────┼────────┤ RX(0) ├──┤ RY(0) ├┤ RZ(0) ├─■───────■──────┤ RX(0) ├┤ RY(0) ├┤ RZ(0) ├┤ RZ(0) ├┤ Z ├┤ Z ├─■───────■──────┤ RZ(0) ├┤ RX(0) ├┤ RY(0) ├┤ RZ(0) ├─■───────■──────┤ RZ(0) ├┤ RY(0) ├┤ RX(0) ├
          └───┘    ┌─┴─┐      ├───────┤  ├───────┤├───────┤ │ZZ(0)  │ZZ(0) ├───────┤├───────┤├───────┤├───────┤├───┤├───┤ │ZZ(0)  │ZZ(0) ├───────┤├───────┤├───────┤├───────┤ │ZZ(0)  │ZZ(0) ├───────┤├───────┤├───────┤
q_3: ──────────────┤ X ├──────┤ RX(0) ├──┤ RY(0) ├┤ RZ(0) ├─■───────■──────┤ RX(0) ├┤ RY(0) ├┤ RZ(0) ├┤ RZ(0) ├┤ Z ├┤ Z ├─■───────■──────┤ RZ(0) ├┤ RX(0) ├┤ RY(0) ├┤ RZ(0) ├─■───────■──────┤ RZ(0) ├┤ RY(0) ├┤ RX(0) ├
                   └───┘      └───────┘  └───────┘└───────┘                └───────┘└───────┘└───────┘└───────┘└───┘└───┘                └───────┘└───────┘└───────┘└───────┘                └───────┘└───────┘└───────┘
c: 2/═══════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════════

In [ ]:
def average_gate_fidelity(circuit_3, num_qubits, parameters, alpha):
    # qreg_q = QuantumRegister(num_qubits*2, 'q_2')
    # creg_c_2 = ClassicalRegister(num_qubits*2, 'c_2')
    # temp_circuit = QuantumCircuit(qreg_q, creg_c_2)

    entanglement_fidelity(circuit_3, num_qubits, parameters, alpha)
    for i in range(num_qubits):
        circuit_3.cx(qreg_q[i], qreg_q[num_qubits + i])
        circuit_3.h(qreg_q[i])

    circuit_3.measure(qreg_q[:], creg_c[:])
    # circuit_3.draw('text', scale = 0.25, plot_barriers = False, fold = -1)
    return circuit_3
        
        
    

In [ ]:
# shots = 2048
# lhst = 0
# for i in range(num_qubits):
#   # temp_q = QuantumRegister(num_qubits*2, 'temp_q')
#   # temp_c = ClassicalRegister(num_qubits*2, 'temp_c')
#   # temp_circuit = QuantumCircuit(temp_q, temp_c)
#   creg_c = ClassicalRegister(num_qubits, 'c')
#   temp_circuit = circuit2()
#   temp_circuit = entanglement_fidelity(temp_circuit, num_qubits, parameters, alpha)
#   temp_circuit.h(qreg_q[i])
#   temp_circuit.cx(qreg_q[i],qreg_q[num_qubits + i])
#   temp_circuit.measure([qreg_q[i], qreg_q[num_qubits + i]], creg_c[:])

#   t_qc = transpile(temp_circuit, backend, optimization_level=3)
#   qobj = assemble(t_qc, shots = shots)
#   job = backend.run(qobj)
#   result = job.result()
#   counts = result.get_counts(temp_circuit)
#   print(counts)
#   # plot_histogram(result.get_counts(temp_circuit))

# #   probs = {}
# #   for output in ['00','01', '10', '11']:
# #       if output in counts:
# #           probs[output] = counts[output]/shots
# #       else:
# #           probs[output] = 0
# #   lhst += probs['00']
# # cost_lhst = 1 - lhst/num_qubits
# temp_circuit.draw('text', scale = 0.25, plot_barriers=False, fold=-1)


In [ ]:
def compute_LHST( num_qubits, parameters, alpha):
  shots = 2048
  lhst = 0
  for i in range(num_qubits):
    # temp_q = QuantumRegister(num_qubits*2, 'temp_q')
    # temp_c = ClassicalRegister(num_qubits*2, 'temp_c')
    # temp_circuit = QuantumCircuit(temp_q, temp_c)
    creg_c = ClassicalRegister(2, 'c')
    temp_circuit = circuit2()
    temp_circuit = entanglement_fidelity(temp_circuit, num_qubits, parameters, alpha)
    temp_circuit.h(qreg_q[i])
    temp_circuit.cx(qreg_q[i],qreg_q[num_qubits + i])
    temp_circuit.measure([qreg_q[i], qreg_q[num_qubits + i]], creg_c[:])


    t_qc = transpile(temp_circuit, backend, optimization_level=3)
    qobj = assemble(t_qc, shots = shots)
    job = backend.run(qobj)

    result = job.result()
    counts = result.get_counts(temp_circuit)
    plot_histogram(counts)
    probs = {}
    for output in ['00','01', '10', '11']:
        if output in counts:
            probs[output] = counts[output]/shots
        else:
            probs[output] = 0
    lhst += probs['00']
  cost_lhst = 1 - lhst/num_qubits 

  return cost_lhst


In [ ]:
# cost_lhst = compute_LHST( num_qubits, parameters, alpha)
# print(cost_lhst)

In [ ]:
# shots = 2048
# avg_gate_fidelity = 0
# d = 2**(num_qubits)

# # temp_q = QuantumRegister(num_qubits*2, 'temp_q')
# # temp_c = ClassicalRegister(num_qubits*2, 'temp_c')
# # temp_circuit = QuantumCircuit(temp_q, temp_c)
# temp_circuit = circuit_3

# temp_circuit = average_gate_fidelity(temp_circuit, num_qubits, parameters, alpha)

# t_qc = transpile(temp_circuit, backend, optimization_level=3)
# qobj = assemble(t_qc, shots = shots)
# job = backend.run(qobj)

# result = job.result()
# counts = result.get_counts(temp_circuit)
# print(counts)
# probs = {}

# output = '0'
# for i in range(num_qubits*2 - 1):
#   output += '0'
    
# if output in counts:
#     probs[output] = counts[output]/shots
# else:
#     probs[output] = 0
  
# cost_hst = 1 - probs[output]
# avg_gate_fidelity = (d*(1 - cost_hst) + 1)/(d + 1)

In [ ]:
def compute_avg_gate_fidelity(circuit_3, num_qubits, parameters, alpha):
  shots = 2048
  avg_gate_fidelity = 0
  d = 2**(num_qubits)

  # temp_q = QuantumRegister(num_qubits*2, 'temp_q')
  # temp_c = ClassicalRegister(num_qubits*2, 'temp_c')
  # temp_circuit = QuantumCircuit(temp_q, temp_c)
  temp_circuit = circuit_3

  temp_circuit = average_gate_fidelity(temp_circuit, num_qubits, parameters, alpha)
  
  t_qc = transpile(temp_circuit, backend, optimization_level=3)
  qobj = assemble(t_qc, shots = shots)
  job = backend.run(qobj)

  result = job.result()
  counts = result.get_counts(temp_circuit)
  probs = {}

  output = '0'
  for i in range(num_qubits*2 - 1):
    output += '0'
      
  if output in counts:
      probs[output] = counts[output]/shots
  else:
      probs[output] = 0
    
  cost_hst = 1 - probs[output]
  avg_gate_fidelity = (d*(1 - cost_hst) + 1)/(d + 1)
  

  return avg_gate_fidelity, temp_circuit


In [ ]:
def cost_threshold( num_qubits, parameters, alpha):
  d = 2**(num_qubits)
  temp_parameters = copy.deepcopy(parameters)
  temp_parameters['step'] = 50
  epsilon_trotter_error = 0
  N = temp_parameters['step']  # no of time the unitaries are exponentiated
  avg_gate_fidelity, temp_circuit = compute_avg_gate_fidelity(circuit3(), num_qubits, temp_parameters, alpha)
  
  c_threshold = (1/num_qubits)*(((1/N)*(((d + 1)*(1 - avg_gate_fidelity))/d)**(0.5) - epsilon_trotter_error)**2)

  return c_threshold, temp_circuit
  

In [ ]:
c_threshold, temp_circuit = cost_threshold(num_qubits, parameters, alpha)
print(c_threshold)
# temp_circuit.draw('text', scale = 0.25, plot_barriers = False, fold = -1)

0.00019521484375


In [ ]:
def W_plus_minus_x(label, alpha):
  
  if label == '+':
    alpha['theta_x'] = alpha['theta_x'] + pi/2
  elif label == '-':
    alpha['theta_x'] = alpha['theta_x'] - pi/2
  
  return alpha

def W_plus_minus_y(label, alpha):
  
  if label == '+':
    alpha['theta_y'] = alpha['theta_y'] + pi/2
  elif label == '-':
    alpha['theta_y'] = alpha['theta_y'] - pi/2
  
  return alpha

def W_plus_minus_z(label, alpha):
  
  if label == '+':
    alpha['theta_z'] = alpha['theta_z'] + pi/2
  elif label == '-':
    alpha['theta_z'] = alpha['theta_z'] - pi/2
  
  return alpha

def W_plus_minus_rzz(label, alpha):
  
  if label == '+':
    alpha['theta_rzz'] = alpha['theta_rzz'] + pi/2
  elif label == '-':
    alpha['theta_rzz'] = alpha['theta_rzz'] - pi/2
  
  return alpha

def D_plus_minus_1(label, alpha):
  
  if label == '+':
    alpha['gamma_1'] = alpha['gamma_1'] + pi/2
  elif label == '-':
    alpha['gamma_1'] = alpha['gamma_1'] - pi/2
  
  return alpha

def D_plus_minus_2(label, alpha):
  
  if label == '+':
    alpha['gamma_2'] = alpha['gamma_2'] + pi/2
  elif label == '-':
    alpha['gamma_2'] = alpha['gamma_2'] - pi/2
  
  return alpha

def D_plus_minus_3(label, alpha):
  
  if label == '+':
    alpha['gamma_3'] = alpha['gamma_3'] + pi/2
  elif label == '-':
    alpha['gamma_3'] = alpha['gamma_3'] - pi/2
  
  return alpha

In [ ]:
def partial_derv_wrt_theta_x( num_qubits, parameters, alpha):
  #####Computing Cost_LHST(U,(W_k+)D(W_dagger))#####
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_x('+', alpha_tmp)
  cost_lhst = compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W_k-)D(W_dagger)) ######
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_x('-', alpha_tmp)
  cost_lhst -= compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D(W_k+_dagger)) #######
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_x('+', alpha_tmp)
  cost_lhst += compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D(W_k-_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_x('-', alpha_tmp)
  cost_lhst -= compute_LHST( num_qubits, parameters, alpha_tmp)

  part_derv = 0.5*(cost_lhst)

  return part_derv

In [ ]:
def partial_derv_wrt_theta_y( num_qubits, parameters, alpha):
  #####Computing Cost_LHST(U,(W_k+)D(W_dagger))#####
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_y('+', alpha_tmp)
  cost_lhst = compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W_k-)D(W_dagger)) ######
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_y('-', alpha_tmp)
  cost_lhst -= compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D(W_k+_dagger)) #######
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_y('+', alpha_tmp)
  cost_lhst += compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D(W_k-_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_y('-', alpha_tmp)
  cost_lhst -= compute_LHST( num_qubits, parameters, alpha_tmp)

  part_derv = 0.5*(cost_lhst)

  return part_derv

In [ ]:
def partial_derv_wrt_theta_z( num_qubits, parameters, alpha):
  #####Computing Cost_LHST(U,(W_k+)D(W_dagger))#####
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_z('+', alpha_tmp)
  cost_lhst = compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W_k-)D(W_dagger)) ######
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_z('-', alpha_tmp)
  cost_lhst -= compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D(W_k+_dagger)) #######
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_z('+', alpha_tmp)
  cost_lhst += compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D(W_k-_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_z('-', alpha_tmp)
  cost_lhst -= compute_LHST( num_qubits, parameters, alpha_tmp)

  part_derv = 0.5*(cost_lhst)

  return part_derv

In [ ]:
def partial_derv_wrt_theta_rzz( num_qubits, parameters, alpha):
  #####Computing Cost_LHST(U,(W_k+)D(W_dagger))#####
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_rzz('+', alpha_tmp)
  cost_lhst = compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W_k-)D(W_dagger)) ######
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_rzz('-', alpha_tmp)
  cost_lhst -= compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D(W_k+_dagger)) #######
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_rzz('+', alpha_tmp)
  cost_lhst += compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D(W_k-_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = W_plus_minus_rzz('-', alpha_tmp)
  cost_lhst -= compute_LHST( num_qubits, parameters, alpha_tmp)

  part_derv = 0.5*(cost_lhst)

  return part_derv

In [ ]:
def partial_derv_wrt_gamma_1( num_qubits, parameters, alpha):
  ##### Computing Cost_LHST(U, (W)D+(W_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = D_plus_minus_1('+', alpha_tmp)
  cost_lhst = compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D-(W_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = D_plus_minus_1('-', alpha_tmp)
  cost_lhst -= compute_LHST(num_qubits, parameters, alpha_tmp)

  part_derv = 0.5*(cost_lhst)

  return part_derv

In [ ]:
def partial_derv_wrt_gamma_2( num_qubits, parameters, alpha):
  ##### Computing Cost_LHST(U, (W)D+(W_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = D_plus_minus_2('+', alpha_tmp)
  cost_lhst = compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D-(W_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = D_plus_minus_2('-', alpha_tmp)
  cost_lhst -= compute_LHST(num_qubits, parameters, alpha_tmp)

  part_derv = 0.5*(cost_lhst)

  return part_derv

In [ ]:
def partial_derv_wrt_gamma_3( num_qubits, parameters, alpha):
  ##### Computing Cost_LHST(U, (W)D+(W_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = D_plus_minus_3('+', alpha_tmp)
  cost_lhst = compute_LHST( num_qubits, parameters, alpha_tmp)

  ##### Computing Cost_LHST(U, (W)D-(W_dagger)) ########
  alpha_tmp = copy.deepcopy(alpha)
  alpha_tmp = D_plus_minus_3('-', alpha_tmp)
  cost_lhst -= compute_LHST(num_qubits, parameters, alpha_tmp)

  part_derv = 0.5*(cost_lhst)

  return part_derv

In [ ]:
def variational_fast_forwarding(num_qubits, parameters, alpha):
  iterations = 1000
  learning_rate = 0.01
  cost_lhst_delta_t = []
  for i in range(iterations):
    cost_lhst = compute_LHST( num_qubits, parameters, alpha)
    if cost_lhst > c_threshold:
      print("\nCost in iteration" ,i,':',cost_lhst,'\n')
      derv_x = learning_rate*partial_derv_wrt_theta_x( num_qubits, parameters, alpha)
      alpha['theta_x'] = alpha['theta_x'] - derv_x
      derv_y = learning_rate*partial_derv_wrt_theta_y( num_qubits, parameters, alpha)
      alpha['theta_y'] = alpha['theta_y'] - derv_y
      derv_z = learning_rate*partial_derv_wrt_theta_z( num_qubits, parameters, alpha)
      alpha['theta_z'] = alpha['theta_z'] - derv_z
      derv_rzz = learning_rate*partial_derv_wrt_theta_rzz( num_qubits, parameters, alpha)
      alpha['theta_rzz'] = alpha['theta_rzz'] - derv_rzz

      derv_1 = learning_rate*partial_derv_wrt_gamma_1( num_qubits, parameters, alpha)
      alpha['gamma_1'] = alpha['gamma_1'] - derv_1
      derv_2 = learning_rate*partial_derv_wrt_gamma_2( num_qubits, parameters, alpha)
      alpha['gamma_2'] = alpha['gamma_2'] - derv_2
      derv_3 = learning_rate*partial_derv_wrt_gamma_3( num_qubits, parameters, alpha)
      alpha['gamma_3'] = alpha['gamma_3'] - derv_3
      print('\nalpha:', alpha,'\n')
      
    elif cost_lhst <= c_threshold:
      print("converged to below threshold")
      return alpha
    cost_lhst_delta_t.append(cost_lhst)


  return alpha, cost_lhst_delta_t 
      

alpha, cost_lhst_delta_t = variational_fast_forwarding(num_qubits, parameters, alpha)



Cost in iteration 0 : 0.7080078125 


alpha: {'theta_x': -6.7138671875e-05, 'theta_y': -0.000142822265625, 'theta_z': 2.0751953125e-05, 'theta_rzz': -6.103515625e-05, 'gamma_1': 0.0001904296875, 'gamma_2': -5.37109375e-05, 'gamma_3': 0.00010009765625000001} 


Cost in iteration 1 : 0.72509765625 


alpha: {'theta_x': 0.0001513671875, 'theta_y': -0.0002978515625, 'theta_z': 0.000140380859375, 'theta_rzz': -0.00014160156250000002, 'gamma_1': 3.2958984375e-05, 'gamma_2': -6.4697265625e-05, 'gamma_3': 5.1269531250000005e-05} 


Cost in iteration 2 : 0.7216796875 


alpha: {'theta_x': 0.00014892578125, 'theta_y': -0.0003125, 'theta_z': 0.00027099609374999997, 'theta_rzz': -0.00018310546875000003, 'gamma_1': -0.00020751953125, 'gamma_2': -3.6621093750000005e-05, 'gamma_3': -1.8310546875000003e-05} 


Cost in iteration 3 : 0.72802734375 


alpha: {'theta_x': 4.5166015625e-05, 'theta_y': -0.00032470703125, 'theta_z': 0.00010253906249999996, 'theta_rzz': -0.00017211914062500002, 'gamma_1': -0.

KeyboardInterrupt: ignored

In [ ]:
print(cost_lhst_delta_t)

NameError: ignored

In [ ]:
# #driver programme for n_half 
# circuit = domain_wall_state(circuit, num_qubits)
# circuit = circuit_for_n_half(circuit, 1, 1, num_qubits, 'xxz-chain-potential')
# circuit.measure(qreg_q, creg_c)
# # circuit.measure(qreg_q[1:10], creg_c[1:10])
# circuit.draw('text', scale = 0.25, plot_barriers = False, fold = -1)

In [ ]:
# %tb
# driver programme for magnetization
circuit = domain_wall_state(circuit, num_qubits)
# circuit = neel_state_preparation(circuit, num_qubits)
circuit = circuit_for_magnetization(circuit, 1, 1, num_qubits)
circuit.measure(qreg_q[:], creg_c[:])
circuit.draw('text', scale = 0.25, plot_barriers = False, fold = -1)

In [ ]:
from qiskit import IBMQ
from qiskit.providers.ibmq import least_busy
shots = 8192

# Load local account information
IBMQ.load_account()
# Get the least busy backend
provider = IBMQ.get_provider(hub='ibm-q')
# backend = least_busy(provider.backends(filters=lambda x: x.configuration().n_qubits >= 2 
#                                        and not x.configuration().simulator 
#                                        and x.status().operational==True))

print("least busy backend: ", backend)
# Run our circuit
t_qc = transpile(circuit, backend, optimization_level=3)
qobj = assemble(t_qc, shots = shots)
job = backend.run(qobj)

In [ ]:
from qiskit.tools.monitor import job_monitor
job_monitor(job)

In [ ]:
result = job.result()
plot_histogram(result.get_counts(circuit))
# print(result.get_counts(circuit))